# 🎨 Notebook 06: Interactive Gradio Demo

## Objective
Create an interactive web interface where users can upload flower images and get instant predictions.

## Features
- Drag-and-drop image upload
- Real-time classification with ViT-Small (98.75% accuracy)
- Top-5 predictions with confidence scores
- Public shareable URL

## Why Gradio?
- Zero frontend code required
- Works directly in Colab
- Creates shareable public links
- Professional UI out of the box

In [3]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.0/23.0 MB 1.4 MB/s  0:00:15m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 1.7 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20/20 [gradio]19/20 [gradio]


In [4]:
# Cell 1: Environment Setup
# ============================================================
# 🎨 Oxford 102 Flowers - Interactive Demo
# Bonus: Gradio Web Interface
# ============================================================
"""
This notebook creates an interactive web demo where users can:
- Upload any flower image
- Get instant predictions with confidence scores
- See top-5 predictions with probabilities
"""

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install -q gradio timm albumentations
    from google.colab import drive
    drive.mount('/content/drive')
    !unzip -q "/content/drive/MyDrive/flowers-cv-submission.zip" -d /content/ 2>/dev/null || true
else:
    pass  # Local environment
    
print("✅ Setup complete")

✅ Setup complete


In [6]:
# Cell 2: Load Model
import torch
import timm
import numpy as np
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import gradio as gr

# Flower class names (Oxford 102)
CLASS_NAMES = [
    "pink primrose", "hard-leaved pocket orchid", "canterbury bells", "sweet pea",
    "english marigold", "tiger lily", "moon orchid", "bird of paradise", "monkshood",
    "globe thistle", "snapdragon", "colt's foot", "king protea", "spear thistle",
    "yellow iris", "globe-flower", "purple coneflower", "peruvian lily", "balloon flower",
    "giant white arum lily", "fire lily", "pincushion flower", "fritillary", "red ginger",
    "grape hyacinth", "corn poppy", "prince of wales feathers", "stemless gentian",
    "artichoke", "sweet william", "carnation", "garden phlox", "love in the mist",
    "mexican aster", "alpine sea holly", "ruby-lipped cattleya", "cape flower",
    "great masterwort", "siam tulip", "lenten rose", "barbeton daisy", "daffodil",
    "sword lily", "poinsettia", "bolero deep blue", "wallflower", "marigold",
    "buttercup", "oxeye daisy", "common dandelion", "petunia", "wild pansy",
    "primula", "sunflower", "pelargonium", "bishop of llandaff", "gaura",
    "geranium", "orange dahlia", "pink-yellow dahlia", "cautleya spicata",
    "japanese anemone", "black-eyed susan", "silverbush", "californian poppy",
    "osteospermum", "spring crocus", "bearded iris", "windflower", "tree poppy",
    "gazania", "azalea", "water lily", "rose", "thorn apple", "morning glory",
    "passion flower", "lotus", "toad lily", "anthurium", "frangipani", "clematis",
    "hibiscus", "columbine", "desert-rose", "tree mallow", "magnolia", "cyclamen",
    "watercress", "canna lily", "hippeastrum", "bee balm", "ball moss", "foxglove",
    "bougainvillea", "camellia", "mallow", "mexican petunia", "bromelia",
    "blanket flower", "trumpet creeper", "blackberry lily"
]

# Load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = timm.create_model("vit_small_patch16_224.augreg_in21k_ft_in1k", num_classes=102)

# Load weights
if IN_COLAB:
    MODEL_PATH = "/content/flowers-cv/artifacts/models/vit_2stage_best.pt"
else:
    MODEL_PATH = "../artifacts/models/vit_2stage_best.pt"

checkpoint = torch.load(MODEL_PATH, map_location=device)

# Handle different checkpoint formats
if isinstance(checkpoint, dict):
    print(f"Checkpoint keys: {checkpoint.keys()}")
    if "model_state_dict" in checkpoint:
        model.load_state_dict(checkpoint["model_state_dict"])
    elif "state_dict" in checkpoint:
        model.load_state_dict(checkpoint["state_dict"])
    elif "model" in checkpoint:
        model.load_state_dict(checkpoint["model"])
    else:
        # Checkpoint dict IS the state dict
        model.load_state_dict(checkpoint)
else:
    model.load_state_dict(checkpoint)

model.to(device)
model.eval()

print(f"✅ Model loaded: ViT-Small (98.75% accuracy)")
print(f"   Parameters: {sum(p.numel() for p in model.parameters()):,}")

Using device: cuda
Checkpoint keys: dict_keys(['cls_token', 'pos_embed', 'patch_embed.proj.weight', 'patch_embed.proj.bias', 'blocks.0.norm1.weight', 'blocks.0.norm1.bias', 'blocks.0.attn.qkv.weight', 'blocks.0.attn.qkv.bias', 'blocks.0.attn.proj.weight', 'blocks.0.attn.proj.bias', 'blocks.0.norm2.weight', 'blocks.0.norm2.bias', 'blocks.0.mlp.fc1.weight', 'blocks.0.mlp.fc1.bias', 'blocks.0.mlp.fc2.weight', 'blocks.0.mlp.fc2.bias', 'blocks.1.norm1.weight', 'blocks.1.norm1.bias', 'blocks.1.attn.qkv.weight', 'blocks.1.attn.qkv.bias', 'blocks.1.attn.proj.weight', 'blocks.1.attn.proj.bias', 'blocks.1.norm2.weight', 'blocks.1.norm2.bias', 'blocks.1.mlp.fc1.weight', 'blocks.1.mlp.fc1.bias', 'blocks.1.mlp.fc2.weight', 'blocks.1.mlp.fc2.bias', 'blocks.2.norm1.weight', 'blocks.2.norm1.bias', 'blocks.2.attn.qkv.weight', 'blocks.2.attn.qkv.bias', 'blocks.2.attn.proj.weight', 'blocks.2.attn.proj.bias', 'blocks.2.norm2.weight', 'blocks.2.norm2.bias', 'blocks.2.mlp.fc1.weight', 'blocks.2.mlp.fc1.bias

In [7]:
# Cell 3: Prediction Function
# Image transform (same as training)
transform = A.Compose([
    A.Resize(256, 256),
    A.CenterCrop(224, 224),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

def predict_flower(image):
    """
    Predict flower species from an image.
    
    Args:
        image: PIL Image or numpy array
        
    Returns:
        Dictionary of class names to confidence scores
    """
    if image is None:
        return {}
    
    # Convert to numpy if needed
    if isinstance(image, Image.Image):
        image = np.array(image)
    
    # Apply transforms
    transformed = transform(image=image)
    x = transformed["image"].unsqueeze(0).to(device)
    
    # Predict
    with torch.no_grad():
        logits = model(x)
        probs = torch.softmax(logits, dim=1)[0]
    
    # Get top-5 predictions
    top5_probs, top5_indices = torch.topk(probs, 5)
    
    # Create output dictionary for Gradio
    results = {
        CLASS_NAMES[idx.item()]: prob.item() 
        for prob, idx in zip(top5_probs, top5_indices)
    }
    
    return results

print("✅ Prediction function ready")

✅ Prediction function ready


In [9]:
# Cell 4: Create Gradio Interface
demo = gr.Interface(
    fn=predict_flower,
    inputs=gr.Image(type="numpy", label="Upload a Flower Image 🌸"),
    outputs=gr.Label(num_top_classes=5, label="Predictions"),
    title="🌸 Oxford 102 Flowers Classifier",
    description="""
    ## About This Model
    - **Architecture**: Vision Transformer (ViT-Small)
    - **Accuracy**: 98.75% on Oxford 102 Flowers test set
    - **Training**: 2-stage fine-tuning with differential learning rates
    
    Upload any flower image to get instant classification!
    
    *Part of AQREIGHT Computer Vision Engineer Assessment*
    """,
    article="""
    ### How It Works
    1. Upload an image of a flower
    2. The model processes it through a Vision Transformer
    3. Get top-5 predictions with confidence scores
    
    ### Model Details
    - Trained on 102 flower species from the Oxford dataset
    - Uses self-attention to capture global flower structure
    - Pretrained on ImageNet-21K (14M images)
    
    ### Tips for Best Results
    - Use clear, well-lit images
    - Center the flower in the frame
    - Single flower works better than bouquets
    """,
    theme=gr.themes.Soft()
)

print("✅ Gradio interface created")


✅ Gradio interface created


In [10]:
# Cell 5: Launch Demo
print("""
╔══════════════════════════════════════════════════════════════════════╗
║                    🚀 LAUNCHING GRADIO DEMO                          ║
╠══════════════════════════════════════════════════════════════════════╣
║                                                                      ║
║  A public URL will be generated that anyone can access!              ║
║  The link is valid for 72 hours.                                     ║
║                                                                      ║
║  Features:                                                           ║
║  ├─ Upload any flower image                                          ║
║  ├─ Get instant predictions                                          ║
║  ├─ See top-5 classes with confidence                                ║
║  └─ Share the link with anyone                                       ║
║                                                                      ║
╚══════════════════════════════════════════════════════════════════════╝
""")

# Launch the demo
demo.launch(
    share=True,  # Creates a public URL (works in Colab!)
    debug=False
)


╔══════════════════════════════════════════════════════════════════════╗
║                    🚀 LAUNCHING GRADIO DEMO                          ║
╠══════════════════════════════════════════════════════════════════════╣
║                                                                      ║
║  A public URL will be generated that anyone can access!              ║
║  The link is valid for 72 hours.                                     ║
║                                                                      ║
║  Features:                                                           ║
║  ├─ Upload any flower image                                          ║
║  ├─ Get instant predictions                                          ║
║  ├─ See top-5 classes with confidence                                ║
║  └─ Share the link with anyone                                       ║
║                                                                      ║
╚══════════════════════════════════════════════════

In [11]:
# Cell 6: Summary
print("""
╔══════════════════════════════════════════════════════════════════════╗
║                    📊 GRADIO DEMO SUMMARY                            ║
╠══════════════════════════════════════════════════════════════════════╣
║                                                                      ║
║  BONUS COMPLETED: Interactive Web Demo ✅                            ║
║                                                                      ║
║  FEATURES IMPLEMENTED:                                               ║
║  ├─ ✅ Drag-and-drop image upload                                    ║
║  ├─ ✅ Real-time ViT-Small inference                                 ║
║  ├─ ✅ Top-5 predictions with confidence bars                        ║
║  ├─ ✅ Public shareable URL                                          ║
║  └─ ✅ Professional UI with documentation                            ║
║                                                                      ║
║  WHY THIS MATTERS:                                                   ║
║  ├─ Stakeholders can test without coding                             ║
║  ├─ Easy to demonstrate model capabilities                           ║
║  ├─ Validates model works on real user inputs                        ║
║  └─ Shows production-readiness mindset                               ║
║                                                                      ║
╚══════════════════════════════════════════════════════════════════════╝
""")


╔══════════════════════════════════════════════════════════════════════╗
║                    📊 GRADIO DEMO SUMMARY                            ║
╠══════════════════════════════════════════════════════════════════════╣
║                                                                      ║
║  BONUS COMPLETED: Interactive Web Demo ✅                            ║
║                                                                      ║
║  FEATURES IMPLEMENTED:                                               ║
║  ├─ ✅ Drag-and-drop image upload                                    ║
║  ├─ ✅ Real-time ViT-Small inference                                 ║
║  ├─ ✅ Top-5 predictions with confidence bars                        ║
║  ├─ ✅ Public shareable URL                                          ║
║  └─ ✅ Professional UI with documentation                            ║
║                                                                      ║
║  WHY THIS MATTERS:                                     